## 文章摘要

In [55]:
import requests
from bs4 import BeautifulSoup
def parseArticle(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml' )
    title = soup.select_one('h1').text
    content = soup.select_one('.ndArticle_margin p').text
    return {'title':title, 'content': content}


news = parseArticle('https://tw.finance.appledaily.com/realtime/20190720/1603069/')

In [56]:
news['title']

'台積電法說定心丸\u3000台股迎產業利多'

In [57]:
news['content'].split('\xa0')

['延續G20會後氛圍，台股傾向反映產業基本面與聯準會降息預期，加權指數量增上攻，雖尚處5月修正套牢位置，且長時間來資金慣以區間操作，萬一點位附近顯有壓力，不過未來基本面景氣復甦，可望帶動架構偏多發展，持續具有表現可能。',
 '富蘭克林華美第一富基金經理人周書玄指出，資金於7月多呈觀望，主要是等待台積電法說會與聯準會利率會議。台積電管理層計畫提高資本支出，意味先進製程技術所引領的成長動能持續，產業營運具有上修空間，也更加穩固市場對5G發展的正面期待。',
 '同時在降息預期壓抑下，目前美國10年期公債殖利率約2.05%，比較之下，台股殖利率對長線資金具有一定吸引力，配合全球寬鬆環境，資金行情將是台股評價面的重要推升動能。',
 '周書玄分析，科技產業景氣受第2季美中貿易談判影響，實質復甦遞延至今年下半年至明年第1季，正面影響包括適度降溫市場過度樂觀風險，且增加第3季股價表現的想像空間。',
 '現階段市場對關稅議題反映鈍化，視美中抗衡習以為常，而聯準會降息機率偏高，可望採行｢預防式降息｣意味暫無顯著降息必要，整體數據未顯示經濟成長有短時間內轉衰退跡象，造就資金大幅認同美中貿易協商、主要央行寬鬆等因素均將保持正向發展，並於反映於盤面漲勢。',
 '考量美中談判依舊存在歧見，空窗期後仍有變數，且降息利多出盡或造成震盪風險，還是須提防其中任一因素，不如預期發展之風險。',
 '富蘭克林華美高科技基金經理人郭修伸表示，台積電法說會後電子族群熱度升高，在占台股比重高的半導體族群獲正面展望帶動下，四大技術領域高速運算、人工智慧、智慧汽車和物聯網，持續為台股中長期成長主軸，重點投資標的包括無線通訊、光通訊、Data Center等相關領域，展望相對明確。',
 '隨美中走向長期對抗，未來一個世界將可能有兩套標準，台廠之於美國具有生產基地分散優勢，之於中國則有潛在美方限制採購風險下，提供零組件的接單契機，有機會在供應鏈重建中受惠。（齊瑞甄／台北報導）']

In [58]:
import jieba, re
dataset = []
dataset.append(news['title'])
#dataset.extend(news['content'].split('\xa0'))
dataset.extend(re.split('[\xa0|。]',news['content']))


In [59]:
corpus = []

for s in dataset:
    corpus.append(' '.join(jieba.cut(s)))

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [61]:
from sklearn.metrics.pairwise import cosine_distances
cs = cosine_distances(X)

In [62]:
cs[0]

array([0.        , 0.93700592, 1.        , 0.80754991, 0.91993592,
       1.        , 1.        , 1.        , 0.92667644, 1.        ,
       1.        , 1.        , 0.91666667, 1.        , 0.88990362,
       1.        , 1.        , 1.        ])

In [63]:
cs[0].argsort()

array([ 0,  3, 14, 12,  4,  8,  1,  6,  7, 16,  9, 10, 11,  2, 13, 15,  5,
       17], dtype=int64)

In [64]:
idx = cs[0].argsort()[1]
idx

3

In [65]:
dataset[0]

'台積電法說定心丸\u3000台股迎產業利多'

In [66]:
dataset[idx]

'富蘭克林華美第一富基金經理人周書玄指出，資金於7月多呈觀望，主要是等待台積電法說會與聯準會利率會議'

In [69]:
import numpy as np
np_titles = np.array(dataset)
summary = '\n'.join(np_titles[cs[0] < 0.9][1:].tolist())
print(summary)

富蘭克林華美第一富基金經理人周書玄指出，資金於7月多呈觀望，主要是等待台積電法說會與聯準會利率會議
富蘭克林華美高科技基金經理人郭修伸表示，台積電法說會後電子族群熱度升高，在占台股比重高的半導體族群獲正面展望帶動下，四大技術領域高速運算、人工智慧、智慧汽車和物聯網，持續為台股中長期成長主軸，重點投資標的包括無線通訊、光通訊、Data Center等相關領域，展望相對明確


## 問答機器人

In [79]:
import requests
from bs4 import BeautifulSoup
res = requests.get('https://www.ikea.com/ms/zh_TW/TW_campaign/faq.html')
soup = BeautifulSoup(res.text)
faq_ary = []
for faq in soup.select('.faq-list li'):
    if faq.select_one('.faq-title') and re.search('[\u4e00-\u9fa5]{2}', faq.select_one('.faq-title').text.strip()):
        question = faq.select_one('.faq-title').text.strip()
        answer = faq.select_one('.faq-answer').text.strip()
        faq_ary.append({'question':question, 'answer':answer})

In [81]:
corpus = []
for rec in faq_ary:
    corpus.append(' '.join(jieba.cut(rec['question'])))

In [84]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [91]:
query = '我該如何取消我的訂單'
q = ' '.join(jieba.cut(query))
Y = vectorizer.transform([q])

In [92]:
cosine_distances(Y,X)[0].argsort()

array([ 50,  51,  73,  95,  83,  87,  57,  27,  64,  53,  76,  96,  13,
        61,  18,  74,  45,   5,  84,  72,   3,   7,  35,  14,  20,  67,
        69, 106,  77,  70,  71, 105,  66, 107,  75,  65, 108,  68,  78,
        81,  80,  99,  98,  97, 101, 102,  94,  93,  92,  91,  90, 103,
        89,  88,  63,  86,  85, 104,  82, 100,  79,   0,  55,  60,  28,
        26,  25,  24,  23,  22,  21,  19,  17,  29,  16,  12,  11,  10,
         9,   8,   6,   4,   2,   1,  15,  30,  31,  32,  59,  58,  56,
       109,  54,  52,  49,  48,  47,  46,  44,  43,  42,  41,  40,  39,
        38,  37,  36,  34,  33,  62, 110], dtype=int64)

In [93]:
faq_ary[50]

{'question': '如何取消訂單?',
 'answer': '若欲取消訂單，請於訂單成立後的隔日晚上九點前致電 IKEA 客服人員；若你於上述時間後才欲取消訂單，則需負擔全額之運送服務費用，煩請特別留意。\n                            \n                                                    請使用隨貨寄送之折讓單、或於網站上下載空白折讓單，填上訂貨單號，並於右下方空白處親筆簽名後寄回至指定的地址；登載有統一編號的客人需列印兩份折讓單，加蓋公司大小章或統一發票專用章。我們會在收到折讓單確認無誤後，扣除相關費用後將剩餘的款項退還給你。\n                                                    ※請點此下載折讓單\n                                                    ※折讓單寄送地址：台中市南屯區向上路二段168號，請註明「IKEA 線上購物」收'}